In [9]:
!pip install AK-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, models
import numpy as np
from pathlib import Path
from collections import defaultdict
from sklearn.model_selection import train_test_split

from AK_SSL.vision import Trainer
# from dataset import ImageDataset

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:
!gdown https://drive.google.com/uc?id=1rKTlddclivfNRtGqF6NzeIXldl1gwHWp

Downloading...
From (original): https://drive.google.com/uc?id=1rKTlddclivfNRtGqF6NzeIXldl1gwHWp
From (redirected): https://drive.google.com/uc?id=1rKTlddclivfNRtGqF6NzeIXldl1gwHWp&confirm=t&uuid=7ae98585-7de6-440d-b386-b403c65723d4
To: /kaggle/working/tiny-imagenet-200.zip
100%|████████████████████████████████████████| 248M/248M [00:03<00:00, 81.0MB/s]


In [6]:
!unzip -q tiny-imagenet-200.zip

In [7]:
!gdown https://drive.google.com/uc?id=1ozkvyC-eZGeLnbZbRM4cVhxfq0tDfNkD

Downloading...
From (original): https://drive.google.com/uc?id=1ozkvyC-eZGeLnbZbRM4cVhxfq0tDfNkD
From (redirected): https://drive.google.com/uc?id=1ozkvyC-eZGeLnbZbRM4cVhxfq0tDfNkD&confirm=t&uuid=cd5775e4-2456-41b1-8468-f47b3f71e176
To: /kaggle/working/save_for_report.zip
100%|███████████████████████████████████████| 1.91G/1.91G [00:17<00:00, 107MB/s]


In [8]:
!unzip -q save_for_report.zip

In [11]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
from pathlib import Path
from collections import defaultdict
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torch.utils.data import DataLoader

class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_size=(128, 128)):
        self.root_dir = Path(root_dir)
        self.target_size = target_size

        if not self.root_dir.exists():
            raise FileNotFoundError(f"Директория {self.root_dir} не существует")

        self.class_folders = sorted([d for d in self.root_dir.iterdir() if d.is_dir()])

        if not self.class_folders:
            raise ValueError(f"Не найдено папок с классами в {self.root_dir}")

        self.class_names = [folder.name for folder in self.class_folders]
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        self.idx_to_class = {idx: class_name for class_name, idx in self.class_to_idx.items()}

        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize(target_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transform

        self.image_paths = []
        self.labels = []
        self.class_to_indices = defaultdict(list)

        self._collect_data()

        print(f"Датасет загружен: {len(self.image_paths)} изображений, {len(self.class_names)} классов")

    def _collect_data(self):
        for class_name, class_idx in self.class_to_idx.items():
            class_folder = self.root_dir / class_name / 'images'

            image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
            image_files = [f for f in class_folder.iterdir()
                          if f.suffix.lower() in image_extensions]

            for img_path in image_files:
                idx = len(self.image_paths)
                self.image_paths.append(img_path)
                self.labels.append(class_idx)
                self.class_to_indices[class_idx].append(idx)


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        try:
            image = Image.open(img_path).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image, label

        except Exception as e:
            print(f"Ошибка загрузки изображения {img_path}: {e}")
            dummy_image = torch.zeros(3, *self.target_size)
            return dummy_image, label

    def get_class_name(self, label):
        return self.idx_to_class.get(label, "Unknown")

    def split_dataset(self, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, random_seed=42):
        np.random.seed(random_seed)
        torch.manual_seed(random_seed)

        indices = list(range(len(self)))
        labels = [self.labels[i] for i in indices]

        train_idx, temp_idx = train_test_split(
            indices,
            train_size=train_ratio,
            random_state=random_seed,
            stratify=labels
        )

        val_test_ratio = val_ratio / (val_ratio + test_ratio)
        temp_labels = [self.labels[i] for i in temp_idx]

        val_idx, test_idx = train_test_split(
            temp_idx,
            train_size=val_test_ratio,
            random_state=random_seed,
            stratify=temp_labels
        )

        train_dataset = Subset(self, train_idx)
        val_dataset = Subset(self, val_idx)
        test_dataset = Subset(self, test_idx)

        return train_dataset, val_dataset, test_dataset

In [12]:
dataset = ImageDataset(
    root_dir="/kaggle/working/tiny-imagenet-200/train",
    target_size=(128, 128)
)

Датасет загружен: 100000 изображений, 200 классов


In [13]:
image, label = dataset[0]
print(f"Image shape: {image.shape}")
print(f"Label: {label} - {dataset.get_class_name(label)}")

train_dataset, val_dataset, test_dataset = dataset.split_dataset(
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)

print(f"\nDataset sizes:")
print(f"Train: {len(train_dataset)}")
print(f"Val: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)

Image shape: torch.Size([3, 128, 128])
Label: 0 - n01443537

Dataset sizes:
Train: 70000
Val: 15000
Test: 15000


In [14]:
backbone = models.resnet50(weights=None)
feature_size = backbone.fc.in_features
backbone.fc = torch.nn.Identity()

backbone = backbone.to("cuda")

In [16]:
# define Trainer

trainer = Trainer(
    method="SimSiam",
    backbone=backbone,
    feature_size=2048,
    image_size=128,
    save_dir="/kaggle/working/content/save_for_report",
    checkpoint_interval=5,
    reload_checkpoint=False,
    # wandb_run=wandb_run,
    device="cuda"
)

----------------AK_SSL: Vision----------------
Number of workers: 4
Device: cuda:0
--------------------------------------
Method: SimSiam
Projection Dimension: 2048
Projection Hidden Dimension: 2048
Prediction Hidden Dimension: 512
Loss: Negative Cosine Simililarity
Transformation: SimCLRViewTransform
Model parameters: 38,199,360
--------------------------------------


In [18]:
# train

trainer.train(
    dataset=train_dataset,
    batch_size=64,
    start_epoch=31,
    epochs=50,
    optimizer="AdamW",
    weight_decay=1e-6,
    learning_rate=1e-3,
)

Pretext Task Model Training:   0%|          | 0/20 [00:00<?, ?epoch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

...

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

In [19]:
# define Trainer

trainer_moco = Trainer(
    method="mocov2",
    backbone=backbone,
    feature_size=2048,
    image_size=128,
    save_dir="/kaggle/working/content/save_for_report",
    checkpoint_interval=5,
    reload_checkpoint=False,
    # wandb_run=wandb_run,
    device="cuda"
)

----------------AK_SSL: Vision----------------
Number of workers: 4
Device: cuda:0
--------------------------------------
Method: mocov2
Projection Dimension: 128
Number of negative keys: 65536
Momentum for updating the key encoder: 0.999
Loss: InfoNCE Loss
Transformation: SimCLRViewTransform
Model parameters: 55,937,664
--------------------------------------


In [20]:
# train

trainer_moco.train(
    dataset=train_dataset,
    batch_size=64,
    start_epoch=20,
    epochs=50,
    optimizer="AdamW",
    weight_decay=1e-6,
    learning_rate=1e-3,
)

Pretext Task Model Training:   0%|          | 0/31 [00:00<?, ?epoch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

...

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]

  0%|          | 0/1093 [00:00<?, ?batch/s]